
    This file is created for some analysis of the following:
        1 - Accuracy of siminets
        2 - Accuracy of database usage.
    caveat: This will be done with a dataset created by participants
    of this project and this dataset is thought to be stronly biased.
    
    First, Importing modules:

In [1]:
from analysis_tools import dataset_scraper

from packages.pipes.collection.feed_disk import FeedFromDiskPipe
from packages.pipes.collection.cleaning import CleaningPipe
from packages.pipes.collection.simi import SimiPipe


    The custom dataset needs to be converted into "tweet" objects which
    have a functionally similar structure as tweepy tweets.
    They are then saved as a list in a pickle file. The reason for doing 
    this is that it is an easy fromat to handle using the 'pipes' in this project. 
    

In [2]:
# // Load 'tweet' objects.
TOPICS = ["Art", "Food", "Incident", "IT", "Nature", "Sport"]  # // Existing topics.
PICKLE_FILE = "./analysis_tools/dummyset.pickle"

dataset_pre_pickle = dataset_scraper.fetch_data_all(
    path="./analysis_tools/Data",
    topics=TOPICS
)

# // Check that everything got loaded, should be 242.
print(len(dataset_pre_pickle))

# // Assign unique user id's. They are functionally used as primary keys in the DB.
dataset_scraper.assign_uid(dataset_pre_pickle)

# // Pickle
dataset_scraper.save(
    data=dataset_pre_pickle,
    filename=PICKLE_FILE
)

242



    At this stage, there should be a dataset at the
    path specified above (when doing pickeling). Next cell will
    set up three pipes to do the following: 
         - Fetching the dataset.
         - Converting to dataobjects and cleaning.
         - Adding siminets to the dataobjects
    

In [3]:
# // Pipe setup
pipe_dsk = FeedFromDiskPipe(
    filepath=PICKLE_FILE,
)
pipe_cln = CleaningPipe(
    previous_pipe=pipe_dsk,
)
pipe_simi = SimiPipe(
    previous_pipe=pipe_cln,
    recursion_level=2 # Higher is slower and usually more accurate.
)
pipes = [pipe_dsk, pipe_cln, pipe_simi]

242cessed 242 amount of objects



    Now that the pipes are set up, they can loop
    through their content and process everything.
    A thing of note; pipes try to keep their internal
    data count (self.output) below a certain amount.
    The default is 200 (self.threshold_output),
    so the data should be moved somewhere when it's
    done (variable below; 'DATA')


In [ ]:
DATA = [] 
while True:
    for pipe in pipes:
        pipe.process()
    # // Move data to DATA when it is processed.
    if pipe_simi.output: 
        DATA.append(
            pipe_simi.output.pop()
        )
    else: # Check for break here, doing it in while statement won't work.
        break
    print(f"Processed object number: {len(DATA)}", end="\r")



    As all dataobjects have a similarity net at this stage
    some analysis can be done. First part of the analysis
    will be by gauging the effectiveness of similarity nets.
    
    This will require some queries which are converted
    into similarity nets. To achieve this, the similarity
    tool inside pipe_simi can be borrowed (to avoid
    loading it again).
    

In [5]:
SIMITOOL = pipe_simi.simitool # // For convenience.

# // Create dict for queries.
QUERY_SIMI = { 
    topic: SIMITOOL.get_similarity_net(
        query=[topic.lower()],
        max_recursion=3
    ) 
        for topic in TOPICS
}
# // Sample a value.
print(QUERY_SIMI.get(TOPICS[0])[0])



['metallic', 0.29787667592366535]



    For convenience, all dataobjects in DATA will be sorted
    into a dictionary with the following format:
        'topic':composite_siminet'
        

In [ ]:
data_dict = {}
for dataobj in DATA:
    name = dataobj.name
    siminet = dataobj.siminet
    if name in data_dict:
        lst = data_dict[name]
        lst.extend(siminet)
    else:
        data_dict[name] = siminet



    At this stage, the actual result calculation will
    be performed. Each siminet of all queries will be
    compared against all siminets of dataobjects and
    the result will be stored in 'RESULT' with the format: 
        what_was_searched:dict(compared_topic:score)


In [13]:
# // Create dict of results.
RESULT = {}
for q_key in QUERY_SIMI:
    q_simi = QUERY_SIMI.get(q_key)
    
    tmp = {} # // Outer dict
    for d_key in data_dict:
        d_simi = data_dict.get(d_key)
        score = SIMITOOL.get_score_compressed_siminet(
            new = q_simi,
            other = d_simi
        )
        # // Progress printout for convenience.
        print(f"{q_key} vs {d_key} = {score}", end='\r')
        tmp[d_key] = score
    RESULT[q_key] = tmp # // Add outer dict to inner.



    At this stage, the results for siminet accuracy
    should be stored in 'RESULT'. The next cell will
    do a printout.
    



In [19]:
# // Calculate padding for printout:
padding = 0
for key in RESULT:
    if len(key) > padding:
        padding = len(key)

# // Printout
for outer_key in RESULT:
    print(f"Query category: '{outer_key}'")
    outer_dict = RESULT.get(outer_key)
    for inner_key in outer_dict:
        whitespace = padding - len(inner_key) + 5
        
        print(f"\t Tweet category '{inner_key}'"
                 f"{whitespace*' '} got score: '{outer_dict.get(inner_key)}'")   



Query category: 'Art'
	 Tweet category ' Art'          got score: '8710.868540873078'
	 Tweet category ' Food'         got score: '1345.6881854633502'
	 Tweet category ' Incident'     got score: '1355.716669787968'
	 Tweet category ' IT'           got score: '2185.552803486575'
	 Tweet category ' Nature'       got score: '2021.637268096208'
	 Tweet category ' Sport'        got score: '1154.329552670322'
Query category: 'Food'
	 Tweet category ' Art'          got score: '677.1686447461447'
	 Tweet category ' Food'         got score: '7644.890272647085'
	 Tweet category ' Incident'     got score: '1202.4390321671979'
	 Tweet category ' IT'           got score: '283.3709708849594'
	 Tweet category ' Nature'       got score: '1481.1989649434875'
	 Tweet category ' Sport'        got score: '308.73552768429164'
Query category: 'Incident'
	 Tweet category ' Art'          got score: '121.11429974436753'
	 Tweet category ' Food'         got score: '146.6687340637046'
	 Tweet category ' Incident




    Calculating percentages:
    
    


In [ ]:
# // Measure percentages:
# Topics reminder: TOPICS = ["Art", "Food", "IT", "Nature", "Sport"]

count_dict = {}

for item in simi_pipe.output:
    score = simi_tool.get_score_compressed_siminet(item.siminet, query_net)
    if count_dict.get(item.name):
        count_dict[item.name] += score
    else:
        count_dict[item.name] = score
    
